In [2]:
import langchain
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')


In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It",groq_api_key = groq_api_key)


In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi My name is Sachit and i am a student")])

AIMessage(content="Hi Sachit! It's nice to meet you. \n\nWhat subject are you studying? 📚  Are you working on any interesting projects right now? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 20, 'total_tokens': 59, 'completion_time': 0.070909091, 'prompt_time': 7.933e-05, 'queue_time': 0.014374877999999999, 'total_time': 0.070988421}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-005548bd-cb97-489e-bf10-969deb24740c-0', usage_metadata={'input_tokens': 20, 'output_tokens': 39, 'total_tokens': 59})

### Message History idea

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi my name is Sachit and i am a student"),
        AIMessage(content="Hello Sachit! It's nice to meet you.  \n\nWhat subject are you studying? What do you find most interesting about it? \n\n"),
        HumanMessage(content="Whats my name and what do i do?")
    ]
)

AIMessage(content="You said your name is Sachit and that you are a student. 😊 \n\nIs there anything else you'd like to tell me about yourself? I'm happy to chat! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 70, 'total_tokens': 112, 'completion_time': 0.076363636, 'prompt_time': 0.002265384, 'queue_time': 0.013001575999999999, 'total_time': 0.07862902}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2b11ff40-5a55-484a-8ba1-501d51b0b514-0', usage_metadata={'input_tokens': 70, 'output_tokens': 42, 'total_tokens': 112})

### Message History

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store= {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [8]:
responce = with_message_history.invoke(
    [
        HumanMessage(content="Hi i am sachit,i am a student")],
        config=config
)

In [9]:
responce.content

"Hello Sachit! \n\nIt's nice to meet you. What are you studying? 😊  \n\n"

In [10]:
with_message_history.invoke(
    [
        HumanMessage(content="whats my name?")],
        config=config
)

AIMessage(content='Your name is Sachit.  \n\nIs there anything else I can help you with? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 55, 'total_tokens': 78, 'completion_time': 0.041818182, 'prompt_time': 0.001840325, 'queue_time': 0.012491885, 'total_time': 0.043658507}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-aedd7e57-4f6e-4167-b288-2f2f6a0a3f2a-0', usage_metadata={'input_tokens': 55, 'output_tokens': 23, 'total_tokens': 78})

In [11]:
config1 = {"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [
        HumanMessage(content="whats my name?")],
        config=config1
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 13, 'total_tokens': 53, 'completion_time': 0.072727273, 'prompt_time': 8.1249e-05, 'queue_time': 0.014572508999999999, 'total_time': 0.072808522}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-73c26cd1-e839-499d-89fd-8540da1c036d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 40, 'total_tokens': 53})

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
promt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpfull assistance. Answer all question based on best of your ability."),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)
chain = promt|model

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sachit")]})

AIMessage(content="Hi Sachit, it's nice to meet you! \n\nHow can I help you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 32, 'total_tokens': 58, 'completion_time': 0.047272727, 'prompt_time': 0.001029417, 'queue_time': 0.012922651, 'total_time': 0.048302144}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-55cbbd8c-ae72-486a-901a-cf5995521b12-0', usage_metadata={'input_tokens': 32, 'output_tokens': 26, 'total_tokens': 58})

In [14]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [15]:
config = {"configurable":{"session_id":"chat3"}}
with_message_history.invoke(
    [
        HumanMessage(content="whats my name")
    ],
    config=config)

AIMessage(content="As an AI, I have no memory of past conversations and no way to know your name.  \n\nIf you'd like to tell me your name, I'm happy to know! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 29, 'total_tokens': 74, 'completion_time': 0.081818182, 'prompt_time': 0.00015098, 'queue_time': 0.013606178, 'total_time': 0.081969162}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dfe4972a-ae63-49a5-a276-fd6b672c045e-0', usage_metadata={'input_tokens': 29, 'output_tokens': 45, 'total_tokens': 74})

In [16]:
## Add more complexity

promt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpfull assistance. Answer all question based on best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)
chain = promt|model

In [17]:
responce = chain.invoke({"messages":[HumanMessage(content="Hi my name is Krish")],"language":"Hindhi"})
responce.content

'नमस्ते कृष्ण!  😊  मुझे बहुत खुशी है कि आपने मेसे बात की। मुझे आपकी मदद करने में बहुत ख़ुशी होगी। आप मुझसे कोई भी सवाल पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा जवाब देने के लिए। \n\n'

In [18]:
# Let wrap this with message history
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
config = {"configurable":{"session_id":"chat4"}}
responce = with_message_history.invoke({"messages":[HumanMessage(content="Hi my name is Krish")],"language":"Hindi"},
                                       config = config)
responce.content

'नमस्ते कृष्ण!  मुझे बहुत खुशी है कि आपने मुझे चाहा।  \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपकी मदद कर सकूं। \n\n'

In [19]:

with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
config = {"configurable":{"session_id":"chat4"}}
responce = with_message_history.invoke({"messages":[HumanMessage(content="Whatsmy name")],"language":"Hindi"},
                                       config = config)
responce.content

'आपका नाम कृष्ण है, जैसा कि आपने खुद मुझे बताया था।  😊  \n'

## Managing Chat history

In [68]:
from langchain_core.messages import SystemMessage, trim_messages # trim_messages help to how many tokens to send to model in history
trimmer = trim_messages(
    max_tokens = 90,
    strategy = "last",
    token_counter=model,
    include_system = True,
    allow_partial = False,
    start_on="human"
)
messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage("hi may name is sachit"),
    AIMessage("Hi!"),
    HumanMessage("I like vanilla ice cream"),
    AIMessage("nice"),
    HumanMessage("whats 2+2"),
    AIMessage("8"),
    HumanMessage("thanks"),
    AIMessage("no probelem"),
    HumanMessage("having fun?"),
    AIMessage("No!"),
]
trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi may name is sachit', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='8', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no probelem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No!', additional_kwargs={}, response_metadata={})]

In [69]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | promt 
    | model 
)
responce = chain.invoke(
    { 
    "messages": messages + [HumanMessage(content="What math probem did i ask and was the answer correct?")],
    "language": "English"
    }
)
responce.content

'You asked: "What is 2+2?" \n\nAnd the answer is correct!  \n\n2 + 2 = 4 \n\n\n\n'

In [70]:
# lets wrap it in message history
from langchain_core.runnables.history import RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "messages",
)
config = {"configurable":{"session_id":"chat4"}}

In [72]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "What is 2+2?" \n\n\n\n'

In [73]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Sachit! 😊  \n'